## Config Iniciais

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install torch

In [ ]:
import os
import transformers
import torch
import getpass
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline


In [ ]:
login(token="_token_")
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
)
                      


In [ ]:
text_generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
)


def get_response(prompt):
  response = text_generator(prompt)
  gen_text = response[0]['generated_text']
  return gen_text

### Exemplos

In [ ]:
prompt = "O que é computação quântica? desejo que a minha resposta venha em portugues"
llama_response = get_response(prompt)
print(llama_response)


# prompt = "Qual o resultado de 7 x 6 - 42?"
# llama_response = get_response(prompt)
# print(llama_response)

# prompt = "Quem foi a primeira pessoa no espaço?"
# llama_response = get_response(prompt)
# print(llama_response)

## templates e Engenharias de Prompt

In [ ]:
def get_response(prompt):
  response = text_generator(prompt)
  gen_text = response[0]['generated_text']
  return gen_text

In [ ]:
prompt = "Quem foi a primeira pessoa no espaço?"
template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "Você entende português?"
template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "O que é ia" #@param {type:"string"}

template  = """<|system|>
você é um assistente útil.

<|user|>
"{}"<|end|>

<|assistant|>
""".format(prompt)
output = get_response(template)
print(output)

## Explorando Engenharia de Prompt

In [ ]:
# prompt = "O que é ia" #@param {type:"string"}
# prompt = "O que é IA? responda em 1 frase"
prompt = "O que é IA? responda em forma de poema"
sys_prompt = "Você é um assistente virtual prestativo, responda as perguntas em português!"
template  = """<|system|>
{}<|end|>

<|user|>
"{}"<|end|>

<|assistant|>
""".format(sys_prompt , prompt)
output = get_response(template)
print(output)

In [ ]:
prompt = "Gere um código em python que escreva a sequência de fibonacci"
sys_prompt = "Você é um programador experiente. Retorne o código requisitado e se quiser forneça explicações breves, caso convenientes"
template  = """<|system|>
{}<|end|>

<|user|>
"{}"<|end|>

<|assistant|>
""".format(sys_prompt , prompt)
output = get_response(template)
print(output)

## Formato de Mensagens

In [ ]:
prompt = "O que é IA?"

msg = [
    
       {"role" : "system" , "content" : "Você é um assistente virtual prestativo, responda as perguntas em português!"},
       {"role" : "user" , "content" : prompt}
]

output = get_response(msg)
print(output)

In [ ]:
prompt = "Liste o nome de 10 cidades famosas da Europa"

msg = [
    
       {"role" : "system" , "content" : "Você é um assistente virtual prestativo, responda as perguntas em português!"},
       {"role" : "user" , "content" : prompt}
]

output = get_response(msg)
print(output)

## Quantização - Bits and bytes config

### - A quantização reduz a precisão dos números usados para representar os parâmetros do modelo, diminuindo o número de memória e os requisitos computacionais

### - Ao invés de usar números de ponto flutuantes de 32 bits(float32), o modelo pode usar 16 , ou até 8 bits.

### - Este processo pode reduzir significativamente o tamanho do modelo e acelerar a inferência sem causar uma perda substancial na precisão


### Benefícios para LLMs:

1 . permite rodar grandes modelos de linguagem em hardware com recursos limitados;

2. Mantém um bom desempenho sem comprometer significativamente a precisão

## Usando o Meta-Llama-3-8B-instruct

In [ ]:
prompt = ("Quem foi a primeira pessoa no espaço?")
messages = [{"role" : "user" , "content" : prompt}]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generate_ids = model.generate(
    model_inputs, 
    max_new_tokens=1000, 
    do_sample=True, 
    pad_token_id=tokenizer.pad_token_id,  
    eos_token_id=tokenizer.eos_token_id
)
decoded = tokenizer.batch_decode(generate_ids)

output = decoded[0]
print(output)